In [ ]:
working_dir = "/data/projects/dschaub/ANCA-GN_transcriptomics"
data_dir = "/data/projects/dschaub/ANCA-GN_transcriptomics/data/single-cell/exploratory/integrated_objects"
save_dir = "/data/projects/dschaub/ANCA-GN_transcriptomics/data/single-cell/exploratory"

In [ ]:
setwd(working_dir)
RhpcBLASctl::blas_set_num_threads(1)
library(Seurat)
library(dplyr)
# library(tidyverse)
library(here)
# library(readxl)
# library(Matrix)
library(IMSBSeuratHelpers)
library(harmony)
library(ggplot2)
library(MuDataSeurat)
# library(sctransform)
library(future)
library(data.table)
library(xlsx)
plan("multicore", workers = 1)
options(future.globals.maxSize = 20 * 1000 * 1024^2) # 20GB
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 200)

In [ ]:
path = paste0(data_dir, "/Harmony_ANCA_27PK27PB_T_clustered.rds")
seu <- readRDS(path)

In [ ]:
head(seu@meta.data)

In [ ]:
# seu@meta.data$cluster_ids <- Idents(object = seu)

In [ ]:
DefaultAssay(seu) <- "RNA"
new.cluster.ids <- c(
    "CD4 naive",
    "CD4/CD8 CM",
    "NKT/CTL",
    "CD4 EM/RM",
    "CD8 EM/RM",
    "CD8 CM/EM",
    "CD4/CD8 stressed",
    "Treg",
    "CD8 naive",
    "Treg",
    "CD8 CM/EM",
    "NKT/gdT",
    "MAIT",
    "CD4/CD8 CM",
    "NKT/gdT",
    "CD4 EM/RM",
    "Prolif."
)
names(new.cluster.ids) <- levels(seu)
seu <- RenameIdents(seu, new.cluster.ids)

# Get current identities
current.idents <- Idents(seu)

# Use new.cluster.ids to replace the identities
new.idents <- new.cluster.ids[current.idents]

seu@meta.data$cell_type_yu <- Idents(object = seu)

In [ ]:
DimPlot(
    object = seu, reduction = "umap", label = TRUE,
    pt.size = 0.1, label.size = 6, repel = T
) + theme(aspect.ratio = 1)

In [ ]:
str(seu)

In [ ]:
seu@assays$RNA@scale.data <- matrix(numeric(), nrow = 0, ncol = 0)
seu@assays$RNA@var.features <- character()
seu@assays$RNA@meta.features <- data.frame(row.names = row.names(seu@assays$RNA@meta.features))
# seu@assays$RNA@key <- "RNA_"
seu@commands <- list()
seu@graphs <- list()
# seu@reductions <- list()
seu@reductions$pca@feature.loadings <- matrix(numeric(), nrow = 0, ncol = 0)
seu@reductions$harmony@feature.loadings <- matrix(numeric(), nrow = 0, ncol = 0)
seu@reductions$harmony@feature.loadings.projected <- matrix(numeric(), nrow = 0, ncol = 0)

In [ ]:
str(seu)

In [ ]:
save_path = paste0(save_dir, "/ANCA_exploratory_27PK27PB_Tcells.h5mu")
WriteH5MU(seu, save_path)